In [1]:
#Cleaning and preprocessing
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
data = pd.io.parsers.read_csv("DataSet.csv")
del data['fnlwgt']
del data['education_num']
del data['relationship']

In [2]:
def Age_Splitting(people):
    AGE = people
    
    if (AGE > 16 and AGE < 25):
        return 'Young'
    if AGE >= 25 and AGE < 35:
        return 'Middle-aged'
    if AGE >= 35 and AGE < 50:
        return 'Senior'
    else:
        return 'Old'

In [3]:
def hours_Splitting(people):
    hours_peer_week = people
    
    if (hours_peer_week > 0 and hours_peer_week < 25):
        return 'Part-time'
    if hours_peer_week >= 25 and hours_peer_week < 40:
        return 'Full-time'
    if hours_peer_week >= 40 and hours_peer_week < 60:
        return 'Over-time'
    else:
        return 'NoLife'

In [4]:
data['workClass'] = data['workClass'].str.replace('?','Other')
data['workClass'] = data['workClass'].str.replace('Without-pay','Other')
data['workClass'] = data['workClass'].str.replace('Never-worked','Other')


data['workClass'] = data['workClass'].str.replace('State-gov','Public')
data['workClass'] = data['workClass'].str.replace('Local-gov','Public')
data['workClass'] = data['workClass'].str.replace('Federal-gov','Public')

data['workClass'] = data['workClass'].str.replace('Self-emp-not-inc','Self-employee')
data['workClass'] = data['workClass'].str.replace('Self-emp-inc','Self-employee')

data['education'] = data['education'].str.replace('12th','Some-HighSchool')
data['education'] = data['education'].str.replace('11th','Some-HighSchool')
data['education'] = data['education'].str.replace('10th','Some-HighSchool')
data['education'] = data['education'].str.replace('7th-8th','Some-MiddleSchool')
data['education'] = data['education'].str.replace('5th-6th','Some-School')
data['education'] = data['education'].str.replace('9th','Some-MiddleSchool')
data['education'] = data['education'].str.replace('1st-4th','Some-School')

data['marital_status'] = data['marital_status'].str.replace('Married-civ-spouse','Married')
data['marital_status'] = data['marital_status'].str.replace('Married-spouse-absent','Married')
data['marital_status'] = data['marital_status'].str.replace('Never-married','Single')

data['native_country'] = data['native_country'].str.replace('?','Other')
data['occupation'] = data['occupation'].str.replace('?','Other')


data['occupation'] = data['occupation'].str.replace('Tech-support','Service')
data['occupation'] = data['occupation'].str.replace('Protective-serv','Service')
data['occupation'] = data['occupation'].str.replace('Priv-house-serv','Service')
data['occupation'] = data['occupation'].str.replace('Other-service','Service')

data['occupation'] = data['occupation'].str.replace('Exec-managerial','Administrative')
data['occupation'] = data['occupation'].str.replace('Adm-clerical','Administrative')
data['occupation'] = data['occupation'].str.replace('Prof-specialty','Administrative')

data['occupation'] = data['occupation'].str.replace('Craft-repair','ManualJob')
data['occupation'] = data['occupation'].str.replace('Farming-fishing','ManualJob')
data['occupation'] = data['occupation'].str.replace('Handlers-cleaners','ManualJob')
data['occupation'] = data['occupation'].str.replace('Machine-op-inspct','ManualJob')

data['occupation'] = data['occupation'].str.replace('Unknown','Other')
data['occupation'] = data['occupation'].str.replace('Armed-Forces','Other')

data['occupation'] = data['occupation'].str.replace('?','Other')


data['Age'] = data['Age'].apply(Age_Splitting)
data['hours_peer_week'] =  data['hours_peer_week'].apply(hours_Splitting)

In [5]:
#PCA
originalData = data.copy()
initialColumns = data.iloc[:,0:7] 
midColumns = data.iloc[:,7:9]
lastColumn = data.iloc[:,9:12] 
from sklearn.preprocessing import StandardScaler
x = midColumns.values
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x)
from sklearn.decomposition import PCA
pca = PCA()
x_pca = pca.fit_transform(x)
x_pca = pd.DataFrame(x_pca)
x_pca.columns = list(midColumns.columns.values)
data = pd.concat([initialColumns, x_pca, lastColumn],sort=True, axis=1)

In [6]:
#validatig dup rows in data set
len(data.duplicated(subset=None, keep='first'))

16281

In [ ]:
#eda
profile = ProfileReport(data, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile

#covariance
Covariance = np.cov(train_data, test_data, bias=True)[0][1]

In [7]:
newData = data.copy()
categorical_feature_mask = newData.dtypes == object
categorical_cols = newData.columns[categorical_feature_mask].tolist()
le = LabelEncoder()
newData[categorical_cols] = newData[categorical_cols].apply(lambda col: le.fit_transform(col))

In [8]:
#correlation matrix
corrMatrix = newData.corr()
corrMatrix

,Age,workClass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_peer_week,native_country,earnings
Age,1.000000,-0.068317,0.126756,0.157451,0.053881,0.026707,-0.055942,1.201278e-02,-2.297928e-02,0.008669,0.030065,-0.050364
workClass,-0.068317,1.000000,-0.076248,-0.148059,-0.108848,0.032926,0.114503,3.045602e-02,7.272853e-02,-0.045751,0.014520,0.144491
education,0.126756,-0.076248,1.000000,0.082899,0.111317,-0.009049,-0.028941,-7.679921e-03,-5.005252e-02,-0.003559,-0.023053,-0.136130
marital_status,0.157451,-0.148059,0.082899,1.000000,0.058895,-0.075905,-0.202073,-8.842433e-03,-7.637753e-02,-0.006400,0.002793,-0.293692
occupation,0.053881,-0.108848,0.111317,0.058895,1.000000,-0.033032,0.003237,-2.036824e-02,-6.899858e-02,-0.016706,-0.009698,-0.155211
race,0.026707,0.032926,-0.009049,-0.075905,-0.033032,1.000000,0.085816,-3.852921e-03,2.184836e-02,-0.006365,0.134607,0.069142
sex,-0.055942,0.114503,-0.028941,-0.202073,0.003237,0.085816,1.000000,-6.268654e-04,6.445452e-02,0.040441,-0.013487,0.211892
capital_gain,0.012013,0.030456,-0.007680,-0.008842,-0.020368,-0.003853,-0.000627,1.000000e+00,9.013197e-15,-0.003893,-0.000367,0.056332
capital_loss,-0.022979,0.072729,-0.050053,-0.076378,-0.068999,0.021848,0.064455,9.013197e-15,1.000000e+00,-0.016282,0.005783,0.261544
hours_peer_week,0.008669,-0.045751,-0.003559,-0.006400,-0.016706,-0.006365,0.040441,-3.893065e-03,-1.628186e-02,1.000000,-0.005171,0.018845


In [9]:
newData['sex'] = newData['sex'].astype('category')
newData['earnings'] = newData['earnings'].astype('category')
newData['hours_peer_week'] = newData['hours_peer_week'].astype('category')
newData['race'] = newData['race'].astype('category')
newData['marital_status'] = newData['marital_status'].astype('category')
newData['workClass'] = newData['workClass'].astype('category')
newData['Age'] = newData['Age'].astype('category')
newData['education'] = newData['education'].astype('category')
newData['native_country'] = newData['native_country'].astype('category')
newData['occupation'] = newData['occupation'].astype('category')

In [10]:
#Classification - Random forest 
np.random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(
    newData.drop("earnings", axis=1),newData['earnings'],
    train_size = 0.7,random_state=42)

rf = RandomForestClassifier()
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2,3],
    'min_samples_leaf': [3,4],
    'min_samples_split': [7, 9, 11],
    'n_estimators': [50, 150, 175, 300]
}
modelRFC = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2)
modelRFC.fit(X_train,y_train)
cross_val_rfc = cross_val_score(modelRFC, X=X_train, y=y_train, cv=10)
R_SquaredRF = modelRFC.score(X_train,y_train)
print("Cross Validation - Accuracy of RFC: ", round(cross_val_rfc.mean() * 100 , 2),"%")
print("Cross Validation - R_Squared of RFC: ", round(R_SquaredRF * 100 , 2),"%")

#Make Predictions
predictionsRFC = modelRFC.predict(X_test)
rmseSVR = sqrt(mean_squared_error(y_test, predictionsRFC))
print(confusion_matrix(y_test, predictionsRFC))
print(classification_report(y_test, predictionsRFC))

Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:   59.0s finished


Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   50.8s
[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:   55.8s finished


Fitting 2 folds for each of 192 candidates, totalling 384 fits


...

Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   46.8s
[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:   51.5s finished


Fitting 2 folds for each of 192 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   44.8s
[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:   49.4s finished


Cross Validation - Accuracy of RFC:  85.89 %
Cross Validation - R_Squared of RFC:  87.8 %
[[3569  168]
 [ 526  622]]
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      3737
           1       0.79      0.54      0.64      1148

    accuracy                           0.86      4885
   macro avg       0.83      0.75      0.78      4885
weighted avg       0.85      0.86      0.85      4885



In [ ]:
#Classification - KNN
np.random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(newData.drop("earnings", axis=1),newData['earnings'],train_size = 0.7,random_state=42)

neighbors = np.arange(1, 9) 
train_accuracy = np.empty(len(neighbors)) 
test_accuracy = np.empty(len(neighbors)) 


for i, k in enumerate(neighbors): 
    knn = KNeighborsClassifier(n_neighbors=k) 
    knn.fit(X_train, y_train) 
      
    # Compute traning and test data accuracy 
    train_accuracy[i] = knn.score(X_train, y_train) 
    test_accuracy[i] = knn.score(X_test, y_test) 
    cross_val_KNN = cross_val_score(knn, X=X_train, y=y_train, cv=10)
    R_SquaredKNN = knn.score(X_train,y_train)
    print("Cross Validation - Accuracy of KNN: ", round(cross_val_KNN.mean() * 100 , 2),"%")
    print("Cross Validation - R_Squared of KNN: ", round(R_SquaredKNN * 100 , 2),"%")

#Make Predictions
predictionsKNN = knn.predict(X_test)
rmseKNN = sqrt(mean_squared_error(y_test, predictionsKNN))
print('KNN - RMSE: %.3f' % rmseKNN)
print(confusion_matrix(y_test, predictionsKNN))
print(classification_report(y_test, predictionsKNN))

    
plt.plot(neighbors, test_accuracy, label = 'Testing dataset Accuracy') 
plt.plot(neighbors, train_accuracy, label = 'Training dataset Accuracy') 
plt.legend() 
plt.xlabel('n_neighbors') 
plt.ylabel('Accuracy') 
plt.show()


In [ ]:
#classification - SVC

np.random.seed(341)

X_train, X_test, y_train, y_test = train_test_split(newData.drop("earnings",
    axis=1),newData['earnings'],train_size = 0.7,random_state=42)


tuned_parameters = [{'kernel': ['linear'], 'gamma': 'auto',
                    'C': [1, 10]}]

 
modelSVC = GridSearchCV(SVC(), tuned_parameters, n_jobs = -1)
modelSVC.fit(X_train,y_train)
cross_val_SVC = cross_val_score(modelSVC, X=X_train, y=y_train, cv=10)
R_SquaredSVC = modelSVC.score(X_train,y_train)
print("Cross Validation - Accuracy of SVC: ", round(cross_val_SVC.mean() * 100 , 2),"%")
print("Cross Validation - R_Squared of SVC: ", round(R_SquaredSVC * 100 , 2),"%")


#Make Predictions
predictionsSVC = modelSVM.predict(X_test)
rmseSVC = sqrt(mean_squared_error(y_test, predictionsSVC))
print('SVC - RMSE: %.3f' % rmseSVC)
print(confusion_matrix(y_test, predictionsSVC))
print(classification_report(y_test, predictionsSVC))

In [ ]:
originalData.head()

In [ ]:

# Define which gender has better payments on credits 
import matplotlib.pyplot as plt
fg = sns.countplot("earnings", hue='sex', data=originalData);
plt.title("Earnings by gender")
plt.xlabel('Earnings')
total = len(originalData["sex"])
for p in fg.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + 0.13
        y = p.get_y() + p.get_height() + 148
        fg.annotate(percentage, (x, y))

plt.show()

In [ ]:
# Define which work class has better earnings
fg = sns.catplot("earnings", hue='workClass', kind="count", data=originalData, ci=None, aspect=1.3);
fg.set_xlabels('Earnings')
fg.set_ylabels('Count')

In [ ]:
# Define which Education has better payments on credits 
fg = sns.countplot(y="earnings", hue="occupation",
            palette="pastel", edgecolor=".6",
            data=originalData);
plt.title("Earnings by occupation")
plt.ylabel('Earnings')
total = len(originalData["occupation"])
for p in fg.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total) + ' - Total :' + str(p.get_width())
        x = p.get_x() + 111
        y = p.get_y()-0.03 + p.get_height()
        fg.annotate(percentage, (x, y))

plt.show()

In [ ]:
# Define which Age group has better earnings
fg = sns.catplot("earnings", hue='Age', kind="count", data=originalData, ci=None, aspect=1.3);
fg.set_xlabels('Amount')
fg.set_ylabels('Count age group')

In [ ]:
# Define which gender with certaing education has better earnings
fg = sns.catplot(x="earnings",hue="sex", col="education" , kind="count", data=originalData, ci=None, aspect=1.3);
fg.set_xlabels('State')
fg.set_ylabels('Count')


In [ ]:
# Define which gender with certaing age has better earnings
fg = sns.catplot(x="earnings",hue="sex", col="Age" , kind="count", data=originalData, ci=None, aspect=1.3);
fg.set_xlabels('Amount')
fg.set_ylabels('Count')


In [ ]:
# Define which gender with certaing hours per week has better earnings
fg = sns.catplot(x="earnings",hue="hours_peer_week", col="Age" , kind="count", data=originalData, ci=None, aspect=1.3);
fg.set_xlabels('Amount')
fg.set_ylabels('Count')

In [ ]:
# Define which gender in an age group has better capital gain
sns.barplot(x="Age", y="capital_gain", hue="sex" , data=originalData)  

In [ ]:
# Define which gender in an age group has the worst capital loss
sns.barplot(x="Age", y="capital_loss", hue="sex" , data=originalData)  

In [ ]:
sns.catplot(x="sex", y="capital_gain",
                hue="hours_peer_week", col="Age",
                data=originalData, kind="boxen",
                height=4, aspect=.7);

In [ ]:
sns.stripplot(x="sex", y="capital_gain", data=originalData,hue="Age",palette="Set2", dodge=True)

In [ ]:
 sns.catplot(x="Age", y="capital_gain",
                hue="workClass", col="marital_status",
                data=originalData, kind="strip",
                height=4, aspect=.7);

In [ ]:
#trend capital gain through ages
sns.pointplot(x="Age", y="capital_gain", data=originalData, capsize=.2, order=['Young', 'Middle-aged', 'Senior', 'Old'])